In [2]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify
import atexit  # 추가

app = Flask(__name__)

# 모델폴더
model_folder = "C:/jupyter/capston/bald/PredictModels"

# 이미지 전처리 함수
def preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((192, 192))
    img = img.convert("RGB")
    img = np.asarray(img)
    img = img.astype('float32') / 255
    return img

# 모델을 사용한 이미지 분류 함수
def predict_image_with_models(image_path, model_folder):
    categories = np.array(["0", "1", "2", "3"])
    scalp_category = ["미세각질", "피지과다", "모낭사이홍반", "모낭홍반/농포", "비듬", "탈모"]

    model_files = [f for f in os.listdir(model_folder) if f.endswith(".h5")]
    alter_predict = []

    image = Image.open(image_path)

    with tf.device("/device:CPU:0"):
        for i, model_file in enumerate(model_files):
            model_path = os.path.join(model_folder, model_file)
            model = load_model(model_path)
            test = []
            test.append(preprocess_image(image_path))
            test = np.array(test)

            predict = model.predict_classes(test)
            predicted_class_index = predict[0]
            predicted_category = categories[predicted_class_index]

            print(f"모델 파일: {model_file}")
            print("예측된 카테고리(" +  scalp_category[i] +  "): " +  predicted_category + "\n")

            alter_predict.append(predicted_category)

    print("alter_predict: ", alter_predict, "\n")
    
    # 여기서부터 피부진단 시작
    # 양호: 배열 안에 값이 전부 0인 경우
    if all(category == '0' for category in alter_predict):
        return "양호"

    # 건성: 배열 첫 번째 값은 3이고 다섯 번째 값이 1이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        alter_predict[4] == '1' and
        all(category != '3' for category in alter_predict[1:4] + alter_predict[5:])
    ):
        return "건성"

    # 지성: 배열 두 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[1] == '3' and
        all(category != '3' for category in alter_predict[2:])
    ):
        return "지성"

    # 민감성: 배열 세 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[2] == '3' and
        all(category != '3' for category in alter_predict[3:])
    ):
        return "민감성"

    # 지루성: 배열 첫 번째 값이 3이고 다섯 번째, 여섯 번째 값이 2~3이고 나머지가 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        (2 <= int(alter_predict[4]) <= 3) and
        (2 <= int(alter_predict[5]) <= 3) and
        all(category != '3' for category in alter_predict[1:4])
    ):
        return "지루성"

    # 염증성: 배열 네 번째 값이 3이고 세 번째 값이 2~3이면서 나머지가 3이 아닌 경우
    if (
        alter_predict[3] == '3' and
        (2 <= int(alter_predict[2]) <= 3)
    ):
        return "염증성"

    # 비듬성: 배열 2, 5번째 값이 2~3이고 나머지 배열 값이 3이 아닌 경우
    if (
        (2 <= int(alter_predict[1]) <= 3) and
        (2 <= int(alter_predict[4]) <= 3) and
        all(category != '3' for category in alter_predict[0:1] + alter_predict[2:4] + alter_predict[5:])
    ):
        return "비듬성"

    # 탈모: 배열 6번째 값이 3이고 나머지 배열 값이 3이 아닌 경우
    if (
        alter_predict[5] == '3' and
        all(category != '3' for category in alter_predict[:5])
    ):
        return "탈모"

    # 기타: 위 조건에 해당하지 않는 경우
    return "기타"

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image part'})

    file = request.files['image']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        # 이미지를 저장할 경로를 지정합니다.
        upload_dir = './upload'
        if not os.path.exists(upload_dir):
            os.makedirs(upload_dir)
        file_path = os.path.join(upload_dir, file.filename)
        file.save(file_path)

        # 이미지 분석 및 진단 결과 얻기
        diagnosis_result = predict_image_with_models(file_path, model_folder)
        return jsonify({'message': diagnosis_result, 'file_path': file_path})

# Flask 애플리케이션 종료 시 호출될 함수 아마 .py에서만 나오는 것일듯
def on_exit():
    print("Flask 애플리케이션이 종료되었습니다.")

# atexit 모듈을 사용하여 on_exit 함수를 등록합니다.
atexit.register(on_exit)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.35.155:5000/ (Press CTRL+C to quit)


In [2]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify
import atexit
from database import save_data_to_database


app = Flask(__name__)

# 모델폴더
model_folder = "C:/jupyter/capston/bald/PredictModels"

# 이미지 전처리 함수
def preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((192, 192))
    img = img.convert("RGB")
    img = np.asarray(img)
    img = img.astype('float32') / 255
    return img

# 모델을 사용한 이미지 분류 함수
def predict_image_with_models(image_path, model_folder):
    categories = np.array(["0", "1", "2", "3"])
    scalp_category = ["미세각질", "피지과다", "모낭사이홍반", "모낭홍반/농포", "비듬", "탈모"]

    model_files = [f for f in os.listdir(model_folder) if f.endswith(".h5")]
    alter_predict = []

    image = Image.open(image_path)

    with tf.device("/device:CPU:0"):
        for i, model_file in enumerate(model_files):
            model_path = os.path.join(model_folder, model_file)
            model = load_model(model_path)
            test = []
            test.append(preprocess_image(image_path))
            test = np.array(test)

            predict = model.predict_classes(test)
            predicted_class_index = predict[0]
            predicted_category = categories[predicted_class_index]

            print(f"모델 파일: {model_file}")
            print("예측된 카테고리(" +  scalp_category[i] +  "): " +  predicted_category + "\n")

            alter_predict.append(predicted_category)

    print("alter_predict: ", alter_predict, "\n")
    
    # 피부진단 결과 계산
    skin_diagnosis = calculate_skin_diagnosis(alter_predict)

    return skin_diagnosis, alter_predict

def calculate_skin_diagnosis(alter_predict):
    # 여기서부터 피부진단 시작
    # 양호: 배열 안에 값이 전부 0인 경우
    if all(category == '0' for category in alter_predict):
        return "양호"

    # 건성: 배열 첫 번째 값은 3이고 다섯 번째 값이 1이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        alter_predict[4] == '1' and
        all(category != '3' for category in alter_predict[1:4] + alter_predict[5:])
    ):
        return "건성"

    # 지성: 배열 두 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[1] == '3' and
        all(category != '3' for category in alter_predict[2:])
    ):
        return "지성"

    # 민감성: 배열 세 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[2] == '3' and
        all(category != '3' for category in alter_predict[3:])
    ):
        return "민감성"

    # 지루성: 배열 첫 번째 값이 3이고 다섯 번째, 여섯 번째 값이 2~3이고 나머지가 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        (2 <= int(alter_predict[4]) <= 3) and
        (2 <= int(alter_predict[5]) <= 3) and
        all(category != '3' for category in alter_predict[1:4])
    ):
        return "지루성"

    # 염증성: 배열 네 번째 값이 3이고 세 번째 값이 2~3이면서 나머지가 3이 아닌 경우
    if (
        alter_predict[3] == '3' and
        (2 <= int(alter_predict[2]) <= 3)
    ):
        return "염증성"

    # 비듬성: 배열 2, 5번째 값이 2~3이고 나머지 배열 값이 3이 아닌 경우
    if (
        (2 <= int(alter_predict[1]) <= 3) and
        (2 <= int(alter_predict[4]) <= 3) and
        all(category != '3' for category in alter_predict[0:1] + alter_predict[2:4] + alter_predict[5:])
    ):
        return "비듬성"

    # 탈모: 배열 6번째 값이 3이고 나머지 배열 값이 3이 아닌 경우
    if (
        alter_predict[5] == '3' and
        all(category != '3' for category in alter_predict[:5])
    ):
        return "탈모"

    # 기타: 위 조건에 해당하지 않는 경우
    return "기타"

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image part'})

    file = request.files['image']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        # 이미지를 저장할 경로를 지정합니다.
        upload_dir = './upload'
        if not os.path.exists(upload_dir):
            os.makedirs(upload_dir)
        file_path = os.path.join(upload_dir, file.filename)
        file.save(file_path)

        # 이미지 분석 및 진단 결과 얻기
        diagnosis_result, alter_predict = predict_image_with_models(file_path, model_folder)
        print(diagnosis_result)
        print(*alter_predict)
        # 진단 결과와 alter_predict를 함께 JSON 응답으로 반환
        response_data = {'message': diagnosis_result, 'symptom1': alter_predict[0], 
                        'symptom2': alter_predict[1], 'symptom3': alter_predict[2],
                        'symptom4': alter_predict[3], 'symptom5': alter_predict[4],
                        'symptom6': alter_predict[5], 'X': ASDFASFD }
        
        # 데이터베이스에 데이터 저장
        save_data_to_database(file_path, diagnosis_result, alter_predict)

        print("Mysql 전송 완료")
        
        return jsonify(response_data)
# Flask 애플리케이션 종료 시 호출될 함수 아마 .py에서만 나오는 것일듯
def on_exit():
    print("Flask 애플리케이션이 종료되었습니다.")

# atexit 모듈을 사용하여 on_exit 함수를 등록합니다.
atexit.register(on_exit)

if __name__ == '__main__':
    app.run(host='192.168.35.33', port=5000, debug=False, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.35.33:5000/ (Press CTRL+C to quit)
C:\Users\USER\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


모델 파일: 1_Corneous_Xception_4.h5
예측된 카테고리(미세각질): 3

모델 파일: 2_Sebum_Xception_1.h5
예측된 카테고리(피지과다): 1

모델 파일: 3_ErFo_Xception_1.h5
예측된 카테고리(모낭사이홍반): 3

모델 파일: 4_Fep_Xception_1.h5
예측된 카테고리(모낭홍반/농포): 3

모델 파일: 5_Dandruff_Xception_1.h5
예측된 카테고리(비듬): 1



[2023-10-01 03:47:20,786] ERROR in app: Exception on /upload [POST]
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\USER\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\USER\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\USER\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\USER\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\USER\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_12624\1491507151.py", line 147, in upl

모델 파일: 6_Hairloss_Xception_1.h5
예측된 카테고리(탈모): 2

alter_predict:  ['3', '1', '3', '3', '1', '2'] 

염증성
3 1 3 3 1 2


In [ ]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify
import atexit
import subprocess
from database import save_data_to_database
import base64

app = Flask(__name__)

# 모델폴더
model_folder = "C:/jupyter/capston/bald/PredictModels"

# 이미지 전처리 함수
def preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((192, 192))
    img = img.convert("RGB")
    img = np.asarray(img)
    img = img.astype('float32') / 255
    return img

# 모델을 사용한 이미지 분류 함수
def predict_image_with_models(image_path, model_folder):
    categories = np.array(["0", "1", "2", "3"])
    scalp_category = ["미세각질", "피지과다", "모낭사이홍반", "모낭홍반/농포", "비듬", "탈모"]

    model_files = [f for f in os.listdir(model_folder) if f.endswith(".h5")]
    alter_predict = []

    predict1 = []
    predict2 = []
    predict3 = []
    predict4 = []
    predict5 = []
    predict6 = []
    image = Image.open(image_path)

    with tf.device("/device:CPU:0"):
        for i, model_file in enumerate(model_files):
            model_path = os.path.join(model_folder, model_file)
            model = load_model(model_path)
            test = []
            test.append(preprocess_image(image_path))
            test = np.array(test)

            
            predict = model.predict(test)
            predicted_class_index = predict[0]
            
            # Print class probabilities
            print(f"Probabilities for model {model_file} ({scalp_category[i]}):")
            for class_index, probability in enumerate(predicted_class_index):
                print(f'Class {class_index}: {probability * 100}%')
                [predict1, predict2, predict3, predict4, predict5, predict6][i].append(round(probability * 100, 2))
                
            print([predict1, predict2, predict3, predict4, predict5, predict6][i])
            predicted_class_index = np.argmax(predicted_class_index)
            predicted_category = categories[predicted_class_index]

            print(f"모델 파일: {model_file}")
            print("예측된 카테고리(" +  scalp_category[i] +  "): " +  predicted_category + "\n")

            alter_predict.append(predicted_category)

    print("alter_predict: ", alter_predict, "\n")
    
    # 피부진단 결과 계산
    skin_diagnosis = calculate_skin_diagnosis(alter_predict)

    return skin_diagnosis, alter_predict, predict1, predict2, predict3, predict4, predict5, predict6

def calculate_skin_diagnosis(alter_predict):
    # 여기서부터 피부진단 시작
    # 양호: 배열 안에 값이 전부 0인 경우
    if all(category == '0' for category in alter_predict):
        return "양호"

    # 건성: 배열 첫 번째 값은 3이고 다섯 번째 값이 1이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        alter_predict[4] == '1' and
        all(category != '3' for category in alter_predict[1:4] + alter_predict[5:])
    ):
        return "건성"

    # 지성: 배열 두 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[1] == '3' and
        all(category != '3' for category in alter_predict[2:])
    ):
        return "지성"

    # 민감성: 배열 세 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[2] == '3' and
        all(category != '3' for category in alter_predict[3:])
    ):
        return "민감성"

    # 지루성: 배열 첫 번째 값이 3이고 다섯 번째, 여섯 번째 값이 2~3이고 나머지가 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        (2 <= int(alter_predict[4]) <= 3) and
        (2 <= int(alter_predict[5]) <= 3) and
        all(category != '3' for category in alter_predict[1:4])
    ):
        return "지루성"

    # 염증성: 배열 네 번째 값이 3이고 세 번째 값이 2~3이면서 나머지가 3이 아닌 경우
    if (
        alter_predict[3] == '3' and
        (2 <= int(alter_predict[2]) <= 3)
    ):
        return "염증성"

    # 비듬성: 배열 2, 5번째 값이 2~3이고 나머지 배열 값이 3이 아닌 경우
    if (
        (2 <= int(alter_predict[1]) <= 3) and
        (2 <= int(alter_predict[4]) <= 3) and
        all(category != '3' for category in alter_predict[0:1] + alter_predict[2:4] + alter_predict[5:])
    ):
        return "비듬성"

    # 탈모: 배열 6번째 값이 3이고 나머지 배열 값이 3이 아닌 경우
    if (
        alter_predict[5] == '3' and
        all(category != '3' for category in alter_predict[:5])
    ):
        return "탈모"

    # 기타: 위 조건에 해당하지 않는 경우
    return "기타"

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image part'})

    file = request.files['image']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        # 이미지를 저장할 경로를 지정합니다.
        upload_dir = './upload'
        if not os.path.exists(upload_dir):
            os.makedirs(upload_dir)
        file_path = os.path.join(upload_dir, file.filename)
        file.save(file_path)

        # 이미지 분석 및 진단 결과 얻기
        diagnosis_result, alter_predict, predict1, predict2, predict3, predict4, predict5, predict6 = predict_image_with_models(file_path, model_folder)
        print(diagnosis_result)
        print(*alter_predict)
        predict1_str = ','.join(map(str, [int(round(x)) for x in predict1]))
        predict2_str = ','.join(map(str, [int(round(x)) for x in predict2]))
        predict3_str = ','.join(map(str, [int(round(x)) for x in predict3]))
        predict4_str = ','.join(map(str, [int(round(x)) for x in predict4]))
        predict5_str = ','.join(map(str, [int(round(x)) for x in predict5]))
        predict6_str = ','.join(map(str, [int(round(x)) for x in predict6]))

        print(predict1_str)
        print(predict2_str)
        print(predict3_str)
        print(predict4_str)
        print(predict5_str)
        print(predict6_str)
        # 진단 결과와 alter_predict를 함께 JSON 응답으로 반환
        response_data = {'message': diagnosis_result,'predict1': predict1_str, 
                        'predict2': predict2_str, 'predict3': predict3_str,
                        'predict4': predict4_str, 'predict5': predict5_str,
                        'predict6': predict6_str }
        
        # 데이터베이스에 데이터 저장
        save_data_to_database(file_path, diagnosis_result, alter_predict)

        print("Mysql 전송 완료")
        
        return jsonify(response_data)
    
@app.route('/wear_wig', methods=['POST'])
def wear_wig():
    try:
        # 이미지 파일 받기
        image = request.files['image']
        # 선택된 가발 값 받기
        selected_wig = int(request.form['selectedWig'])

        source_path = './unprocessed/source.jpg'
        image.save(source_path)  # 이미지를 지정된 경로에 저장
        
        # 이곳에서 이미지 처리 및 가발 적용 로직을 구현
        script_path = "align_face.py"
        subprocess.run(["python", script_path])
        
        # hair_synthesis.py 스크립트 실행
        hair_synthesis_script = "hair_synthesis.py"
        im_path1 = "source.png"
        im_path2 = "target" + str(selected_wig + 1) + ".png"
        print(im_path2)
        im_path3 = "source.png"
        sign = "realistic"
        smooth = "5"
        
        subprocess.run(["python", hair_synthesis_script, "--im_path1", im_path1, "--im_path2", im_path2, "--im_path3", im_path3, "--sign", sign, "--smooth", smooth])
        
        synthesis_output = "./output/source_target" + str(selected_wig + 1) + "_source_realistic.png"
        
        # 이미지를 바이트로 읽어옴
        with open(synthesis_output, 'rb') as img_file:
            img_bytes = img_file.read()

        # 이미지를 Base64로 인코딩
        img_base64 = base64.b64encode(img_bytes).decode('utf-8')
        
        # 응답 데이터 생성
        response_data = {
            'result': 'number' + str(selected_wig + 1) + 'selected',
            'image_result': img_base64  # 이미지를 Base64로 인코딩하여 추가
        }
        print("전부 완료함")
        return jsonify(response_data)
    
    except Exception as e:
        return str(e), 500

if __name__ == '__main__':
    app.run(host='192.168.35.216', port=5000, debug=False, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.35.216:5000
Press CTRL+C to quit


target2.png


In [1]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify
import atexit
import subprocess
from database import save_data_to_database
import base64
from multiprocessing import Process
from external_command_runner import run_external_command

app = Flask(__name__)

# 모델폴더
model_folder = "C:/jupyter/capston/bald/PredictModels"

# 이미지 전처리 함수
def preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((192, 192))
    img = img.convert("RGB")
    img = np.asarray(img)
    img = img.astype('float32') / 255
    return img

# 모델을 사용한 이미지 분류 함수
def predict_image_with_models(image_path, model_folder):
    categories = np.array(["0", "1", "2", "3"])
    scalp_category = ["미세각질", "피지과다", "모낭사이홍반", "모낭홍반/농포", "비듬", "탈모"]

    model_files = [f for f in os.listdir(model_folder) if f.endswith(".h5")]
    alter_predict = []

    predict1 = []
    predict2 = []
    predict3 = []
    predict4 = []
    predict5 = []
    predict6 = []
    image = Image.open(image_path)

    with tf.device("/device:CPU:0"):
        for i, model_file in enumerate(model_files):
            model_path = os.path.join(model_folder, model_file)
            model = load_model(model_path)
            test = []
            test.append(preprocess_image(image_path))
            test = np.array(test)

            
            predict = model.predict(test)
            predicted_class_index = predict[0]
            
            # Print class probabilities
            print(f"Probabilities for model {model_file} ({scalp_category[i]}):")
            for class_index, probability in enumerate(predicted_class_index):
                print(f'Class {class_index}: {probability * 100}%')
                [predict1, predict2, predict3, predict4, predict5, predict6][i].append(round(probability * 100, 2))
                
            print([predict1, predict2, predict3, predict4, predict5, predict6][i])
            predicted_class_index = np.argmax(predicted_class_index)
            predicted_category = categories[predicted_class_index]

            print(f"모델 파일: {model_file}")
            print("예측된 카테고리(" +  scalp_category[i] +  "): " +  predicted_category + "\n")

            alter_predict.append(predicted_category)

    print("alter_predict: ", alter_predict, "\n")
    
    # 피부진단 결과 계산
    skin_diagnosis = calculate_skin_diagnosis(alter_predict)

    return skin_diagnosis, alter_predict, predict1, predict2, predict3, predict4, predict5, predict6

def calculate_skin_diagnosis(alter_predict):
    # 여기서부터 피부진단 시작
    # 양호: 배열 안에 값이 전부 0인 경우
    if all(category == '0' for category in alter_predict):
        return "양호"

    # 건성: 배열 첫 번째 값은 3이고 다섯 번째 값이 1이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        alter_predict[4] == '1' and
        all(category != '3' for category in alter_predict[1:4] + alter_predict[5:])
    ):
        return "건성"

    # 지성: 배열 두 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[1] == '3' and
        all(category != '3' for category in alter_predict[2:])
    ):
        return "지성"

    # 민감성: 배열 세 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[2] == '3' and
        all(category != '3' for category in alter_predict[3:])
    ):
        return "민감성"

    # 지루성: 배열 첫 번째 값이 3이고 다섯 번째, 여섯 번째 값이 2~3이고 나머지가 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        (2 <= int(alter_predict[4]) <= 3) and
        (2 <= int(alter_predict[5]) <= 3) and
        all(category != '3' for category in alter_predict[1:4])
    ):
        return "지루성"

    # 염증성: 배열 네 번째 값이 3이고 세 번째 값이 2~3이면서 나머지가 3이 아닌 경우
    if (
        alter_predict[3] == '3' and
        (2 <= int(alter_predict[2]) <= 3)
    ):
        return "염증성"

    # 비듬성: 배열 2, 5번째 값이 2~3이고 나머지 배열 값이 3이 아닌 경우
    if (
        (2 <= int(alter_predict[1]) <= 3) and
        (2 <= int(alter_predict[4]) <= 3) and
        all(category != '3' for category in alter_predict[0:1] + alter_predict[2:4] + alter_predict[5:])
    ):
        return "비듬성"

    # 탈모: 배열 6번째 값이 3이고 나머지 배열 값이 3이 아닌 경우
    if (
        alter_predict[5] == '3' and
        all(category != '3' for category in alter_predict[:5])
    ):
        return "탈모"

    # 기타: 위 조건에 해당하지 않는 경우
    return "기타"

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image part'})

    file = request.files['image']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        # 이미지를 저장할 경로를 지정합니다.
        upload_dir = './upload'
        if not os.path.exists(upload_dir):
            os.makedirs(upload_dir)
        file_path = os.path.join(upload_dir, file.filename)
        file.save(file_path)

        # 이미지 분석 및 진단 결과 얻기
        diagnosis_result, alter_predict, predict1, predict2, predict3, predict4, predict5, predict6 = predict_image_with_models(file_path, model_folder)
        print(diagnosis_result)
        print(*alter_predict)
        predict1_str = ','.join(map(str, [int(round(x)) for x in predict1]))
        predict2_str = ','.join(map(str, [int(round(x)) for x in predict2]))
        predict3_str = ','.join(map(str, [int(round(x)) for x in predict3]))
        predict4_str = ','.join(map(str, [int(round(x)) for x in predict4]))
        predict5_str = ','.join(map(str, [int(round(x)) for x in predict5]))
        predict6_str = ','.join(map(str, [int(round(x)) for x in predict6]))

        print(predict1_str)
        print(predict2_str)
        print(predict3_str)
        print(predict4_str)
        print(predict5_str)
        print(predict6_str)
        # 진단 결과와 alter_predict를 함께 JSON 응답으로 반환
        response_data = {'message': diagnosis_result,'predict1': predict1_str, 
                        'predict2': predict2_str, 'predict3': predict3_str,
                        'predict4': predict4_str, 'predict5': predict5_str,
                        'predict6': predict6_str }
        
        # 데이터베이스에 데이터 저장
        save_data_to_database(file_path, diagnosis_result, alter_predict)

        print("Mysql 전송 완료")
        
        return jsonify(response_data)
    
@app.route('/wear_wig', methods=['POST'])
def wear_wig():
    try:
        # 이미지 파일 받기
        image = request.files['image']
        # 선택된 가발 값 받기
        selected_wig = int(request.form['selectedWig'])

        source_path = './unprocessed/source.jpg'
        image.save(source_path)  # 이미지를 지정된 경로에 저장
        
        # 이곳에서 이미지 처리 및 가발 적용 로직을 구현
        script_path = "align_face.py"

        # 별도 프로세스로 외부 명령 실행
        process1 = Process(target=run_external_command, args=(["python", script_path],))
        process1.start()

        # hair_synthesis.py 스크립트 실행
        hair_synthesis_script = "hair_synthesis.py"
        im_path1 = "source.png"
        im_path2 = "target" + str(selected_wig + 1) + ".png"
        print(im_path2)
        im_path3 = "source.png"
        sign = "realistic"
        smooth = "5"

        # 별도 프로세스로 외부 명령 실행
        process2 = Process(target=run_external_command, args=(["python", hair_synthesis_script, "--im_path1", im_path1, "--im_path2", im_path2, "--im_path3", im_path3, "--sign", sign, "--smooth", smooth],))
        process2.start()
        process2.join()  # 프로세스 완료 대기
        
        synthesis_output = "./output/source_target" + str(selected_wig + 1) + "_source_realistic.png"
        
        # 이미지를 바이트로 읽어옴
        with open(synthesis_output, 'rb') as img_file:
            img_bytes = img_file.read()

        # 이미지를 Base64로 인코딩
        img_base64 = base64.b64encode(img_bytes).decode('utf-8')
        
        # 응답 데이터 생성
        response_data = {
            'result': 'number' + str(selected_wig + 1) + 'selected',
            'image_result': img_base64  # 이미지를 Base64로 인코딩하여 추가
        }
        print("전부 완료함")
        return jsonify(response_data)
    
    except Exception as e:
        return str(e), 500
    
if __name__ == '__main__':
    app.run(host='192.168.35.216', port=5000, debug=False, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.35.216:5000
Press CTRL+C to quit


target2.png


192.168.35.216 - - [16/Oct/2023 12:04:47] "POST /wear_wig HTTP/1.1" 200 -


전부 완료함


In [1]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify, send_file
import atexit
from database import save_data_to_database
import base64
from subprocess_code import prepro_image, synthesis_image
import binascii

app = Flask(__name__)

# 모델폴더
model_folder = "C:/jupyter/capston/bald/PredictModels"

# 이미지 전처리 함수
def preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((192, 192))
    img = img.convert("RGB")
    img = np.asarray(img)
    img = img.astype('float32') / 255
    return img

# 모델을 사용한 이미지 분류 함수
def predict_image_with_models(image_path, model_folder):
    categories = np.array(["0", "1", "2", "3"])
    scalp_category = ["미세각질", "피지과다", "모낭사이홍반", "모낭홍반/농포", "비듬", "탈모"]

    model_files = [f for f in os.listdir(model_folder) if f.endswith(".h5")]
    alter_predict = []

    predict1 = []
    predict2 = []
    predict3 = []
    predict4 = []
    predict5 = []
    predict6 = []
    image = Image.open(image_path)

    with tf.device("/device:CPU:0"):
        for i, model_file in enumerate(model_files):
            model_path = os.path.join(model_folder, model_file)
            model = load_model(model_path)
            test = []
            test.append(preprocess_image(image_path))
            test = np.array(test)

            
            predict = model.predict(test)
            predicted_class_index = predict[0]
            
            # Print class probabilities
            print(f"Probabilities for model {model_file} ({scalp_category[i]}):")
            for class_index, probability in enumerate(predicted_class_index):
                print(f'Class {class_index}: {probability * 100}%')
                [predict1, predict2, predict3, predict4, predict5, predict6][i].append(round(probability * 100, 2))
                
            print([predict1, predict2, predict3, predict4, predict5, predict6][i])
            predicted_class_index = np.argmax(predicted_class_index)
            predicted_category = categories[predicted_class_index]

            print(f"모델 파일: {model_file}")
            print("예측된 카테고리(" +  scalp_category[i] +  "): " +  predicted_category + "\n")

            alter_predict.append(predicted_category)

    print("alter_predict: ", alter_predict, "\n")
    
    # 피부진단 결과 계산
    skin_diagnosis = calculate_skin_diagnosis(alter_predict)

    return skin_diagnosis, alter_predict, predict1, predict2, predict3, predict4, predict5, predict6

def calculate_skin_diagnosis(alter_predict):
    # 여기서부터 피부진단 시작
    # 양호: 배열 안에 값이 전부 0인 경우
    if all(category == '0' for category in alter_predict):
        return "양호"

    # 건성: 배열 첫 번째 값은 3이고 다섯 번째 값이 1이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        alter_predict[4] == '1' and
        all(category != '3' for category in alter_predict[1:4] + alter_predict[5:])
    ):
        return "건성"

    # 지성: 배열 두 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[1] == '3' and
        all(category != '3' for category in alter_predict[2:])
    ):
        return "지성"

    # 민감성: 배열 세 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[2] == '3' and
        all(category != '3' for category in alter_predict[3:])
    ):
        return "민감성"

    # 지루성: 배열 첫 번째 값이 3이고 다섯 번째, 여섯 번째 값이 2~3이고 나머지가 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        (2 <= int(alter_predict[4]) <= 3) and
        (2 <= int(alter_predict[5]) <= 3) and
        all(category != '3' for category in alter_predict[1:4])
    ):
        return "지루성"

    # 염증성: 배열 네 번째 값이 3이고 세 번째 값이 2~3이면서 나머지가 3이 아닌 경우
    if (
        alter_predict[3] == '3' and
        (2 <= int(alter_predict[2]) <= 3)
    ):
        return "염증성"

    # 비듬성: 배열 2, 5번째 값이 2~3이고 나머지 배열 값이 3이 아닌 경우
    if (
        (2 <= int(alter_predict[1]) <= 3) and
        (2 <= int(alter_predict[4]) <= 3) and
        all(category != '3' for category in alter_predict[0:1] + alter_predict[2:4] + alter_predict[5:])
    ):
        return "비듬성"

    # 탈모: 배열 6번째 값이 3이고 나머지 배열 값이 3이 아닌 경우
    if (
        alter_predict[5] == '3' and
        all(category != '3' for category in alter_predict[:5])
    ):
        return "탈모"

    # 기타: 위 조건에 해당하지 않는 경우
    return "기타"

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image part'})

    file = request.files['image']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        # 이미지를 저장할 경로를 지정합니다.
        upload_dir = './upload'
        if not os.path.exists(upload_dir):
            os.makedirs(upload_dir)
        file_path = os.path.join(upload_dir, file.filename)
        file.save(file_path)

        # 이미지 분석 및 진단 결과 얻기
        diagnosis_result, alter_predict, predict1, predict2, predict3, predict4, predict5, predict6 = predict_image_with_models(file_path, model_folder)
        print(diagnosis_result)
        print(*alter_predict)
        predict1_str = ','.join(map(str, [int(round(x)) for x in predict1]))
        predict2_str = ','.join(map(str, [int(round(x)) for x in predict2]))
        predict3_str = ','.join(map(str, [int(round(x)) for x in predict3]))
        predict4_str = ','.join(map(str, [int(round(x)) for x in predict4]))
        predict5_str = ','.join(map(str, [int(round(x)) for x in predict5]))
        predict6_str = ','.join(map(str, [int(round(x)) for x in predict6]))

        print(predict1_str)
        print(predict2_str)
        print(predict3_str)
        print(predict4_str)
        print(predict5_str)
        print(predict6_str)
        # 진단 결과와 alter_predict를 함께 JSON 응답으로 반환
        response_data = {'message': diagnosis_result,'predict1': predict1_str, 
                        'predict2': predict2_str, 'predict3': predict3_str,
                        'predict4': predict4_str, 'predict5': predict5_str,
                        'predict6': predict6_str }
        
        # 데이터베이스에 데이터 저장
        save_data_to_database(file_path, diagnosis_result, alter_predict)

        print("Mysql 전송 완료")
        
        return jsonify(response_data)
    
selected_wig = 1 # default

@app.route('/wear_wig', methods=['POST'])
def wear_wig():
    # 이미지 파일 받기
    image = request.files['image']
    # 선택된 가발 값 받기
    selected_wig = int(request.form['selectedWig'])

    source_path = './unprocessed/source.jpg'
    image.save(source_path)  # 이미지를 지정된 경로에 저장
        
    # 이곳에서 이미지 처리 및 가발 적용 로직을 구현
    script_path = "align_face.py"
        
    result1 = prepro_image(script_path)
    print(result1)
        
    # hair_synthesis.py 스크립트 실행
    hair_synthesis_script = "hair_synthesis.py"
    im_path1 = "source.png"
    im_path2 = "target" + str(selected_wig + 1) + ".png"
    print(im_path2)
    im_path3 = "source.png"
    sign = "realistic"
    smooth = "5"
        
    result2 = synthesis_image(hair_synthesis_script, im_path1, im_path2, im_path3, sign, smooth)
    print(result2)
        
    synthesis_output = "./output/source_target" + str(selected_wig + 1) + "_source_realistic.png"
        
    print("가발 합성 최종 완료")
    return "이미지 처리 완료"
    
from PIL import Image

@app.route('/result_wig', methods=['GET', 'POST'])
def result_wig():
    print(selected_wig)
    synthesis_output = "./output/source_target" + str(selected_wig + 1) + "_source_realistic.png"
    
    # PNG 이미지를 열고 JPG로 변환
    with Image.open(synthesis_output) as img:
        img = img.convert('RGB')
        jpg_output = synthesis_output.replace(".png", ".jpg")
        img.save(jpg_output, "JPEG")
    
    # JPG 파일을 클라이언트로 전송
    return send_file(jpg_output, mimetype='image/jpeg')

# 500번 오류를 처리하는 예외 핸들러
@app.errorhandler(500)
def internal_server_error(e):
    return "Internal Server Error: " + str(e), 500

if __name__ == '__main__':
    app.run(host='192.168.35.216', port=5000, debug=False, use_reloader=False, threaded=True)

c:\jupyter\capston\bald
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.35.216:5000
Press CTRL+C to quit
192.168.35.216 - - [16/Oct/2023 15:16:42] "GET /result_wig HTTP/1.1" 200 -


1


In [1]:
import os
import torch
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.models import load_model  # 변경된 부분
from flask import Flask, request, jsonify, send_file
import atexit
from database import save_data_to_database
from subprocess_code import prepro_image, synthesis_image

app = Flask(__name__)

# 모델폴더
model_folder = "C:/jupyter/capston/bald/PredictModels"

# 이미지 전처리 함수
def preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((192, 192))
    img = img.convert("RGB")
    img = np.asarray(img)
    img = img.astype('float32') / 255
    return img

# 모델을 사용한 이미지 분류 함수
def predict_image_with_models(image_path, model_folder):
    categories = np.array(["0", "1", "2", "3"])
    scalp_category = ["미세각질", "피지과다", "모낭사이홍반", "모낭홍반/농포", "비듬", "탈모"]

    model_files = [f for f in os.listdir(model_folder) if f.endswith(".h5")]
    print(model_files)
    alter_predict = []

    predict1 = []
    predict2 = []
    predict3 = []
    predict4 = []
    predict5 = []
    predict6 = []
    image = Image.open(image_path)

    with tf.device("/device:CPU:0"):
        for i, model_file in enumerate(model_files):
            print(os.getcwd())
            print(model_file)
            model_path = model_folder + '/' + model_file
            print(model_path)
            model = load_model(model_path)
            test = []
            test.append(preprocess_image(image_path))
            test = np.array(test)

            
            predict = model.predict(test)
            predicted_class_index = predict[0]
            
            # Print class probabilities
            print(f"Probabilities for model {model_file} ({scalp_category[i]}):")
            for class_index, probability in enumerate(predicted_class_index):
                print(f'Class {class_index}: {probability * 100}%')
                [predict1, predict2, predict3, predict4, predict5, predict6][i].append(round(probability * 100, 2))
                
            print([predict1, predict2, predict3, predict4, predict5, predict6][i])
            predicted_class_index = np.argmax(predicted_class_index)
            predicted_category = categories[predicted_class_index]

            print(f"모델 파일: {model_file}")
            print("예측된 카테고리(" +  scalp_category[i] +  "): " +  predicted_category + "\n")

            alter_predict.append(predicted_category)

    print("alter_predict: ", alter_predict, "\n")
    
    # 피부진단 결과 계산
    skin_diagnosis = calculate_skin_diagnosis(alter_predict)

    return skin_diagnosis, alter_predict, predict1, predict2, predict3, predict4, predict5, predict6

def calculate_skin_diagnosis(alter_predict):
    # 여기서부터 피부진단 시작
    # 양호: 배열 안에 값이 전부 0인 경우
    if all(category == '0' for category in alter_predict):
        return "양호"

    # 건성: 배열 첫 번째 값은 3이고 다섯 번째 값이 1이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        alter_predict[4] == '1' and
        all(category != '3' for category in alter_predict[1:4] + alter_predict[5:])
    ):
        return "건성"

    # 지성: 배열 두 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[1] == '3' and
        all(category != '3' for category in alter_predict[2:])
    ):
        return "지성"

    # 민감성: 배열 세 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[2] == '3' and
        all(category != '3' for category in alter_predict[3:])
    ):
        return "민감성"

    # 지루성: 배열 첫 번째 값이 3이고 다섯 번째, 여섯 번째 값이 2~3이고 나머지가 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        (2 <= int(alter_predict[4]) <= 3) and
        (2 <= int(alter_predict[5]) <= 3) and
        all(category != '3' for category in alter_predict[1:4])
    ):
        return "지루성"

    # 염증성: 배열 네 번째 값이 3이고 세 번째 값이 2~3이면서 나머지가 3이 아닌 경우
    if (
        alter_predict[3] == '3' and
        (2 <= int(alter_predict[2]) <= 3)
    ):
        return "염증성"

    # 비듬성: 배열 2, 5번째 값이 2~3이고 나머지 배열 값이 3이 아닌 경우
    if (
        (2 <= int(alter_predict[1]) <= 3) and
        (2 <= int(alter_predict[4]) <= 3) and
        all(category != '3' for category in alter_predict[0:1] + alter_predict[2:4] + alter_predict[5:])
    ):
        return "비듬성"

    # 탈모: 배열 6번째 값이 3이고 나머지 배열 값이 3이 아닌 경우
    if (
        alter_predict[5] == '3' and
        all(category != '3' for category in alter_predict[:5])
    ):
        return "탈모"

    # 기타: 위 조건에 해당하지 않는 경우
    return "기타"

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image part'})

    file = request.files['image']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        # 이미지를 저장할 경로를 지정합니다.
        upload_dir = './upload'
        if not os.path.exists(upload_dir):
            os.makedirs(upload_dir)
        file_path = os.path.join(upload_dir, file.filename)
        file.save(file_path)

        # 이미지 분석 및 진단 결과 얻기
        diagnosis_result, alter_predict, predict1, predict2, predict3, predict4, predict5, predict6 = predict_image_with_models(file_path, model_folder)
        print(diagnosis_result)
        print(*alter_predict)
        predict1_str = ','.join(map(str, [int(round(x)) for x in predict1]))
        predict2_str = ','.join(map(str, [int(round(x)) for x in predict2]))
        predict3_str = ','.join(map(str, [int(round(x)) for x in predict3]))
        predict4_str = ','.join(map(str, [int(round(x)) for x in predict4]))
        predict5_str = ','.join(map(str, [int(round(x)) for x in predict5]))
        predict6_str = ','.join(map(str, [int(round(x)) for x in predict6]))

        print(predict1_str)
        print(predict2_str)
        print(predict3_str)
        print(predict4_str)
        print(predict5_str)
        print(predict6_str)
        # 진단 결과와 alter_predict를 함께 JSON 응답으로 반환
        response_data = {'message': diagnosis_result,'predict1': predict1_str, 
                        'predict2': predict2_str, 'predict3': predict3_str,
                        'predict4': predict4_str, 'predict5': predict5_str,
                        'predict6': predict6_str }
        
        # 데이터베이스에 데이터 저장
        save_data_to_database(file_path, diagnosis_result, alter_predict)

        print("Mysql 전송 완료")
        
        return jsonify(response_data)
    
selected_wig = 0 # default

@app.route('/wear_wig', methods=['POST'])
def wear_wig():
    print(torch.cuda.is_available())
    os.environ["CUDA_VISIBLE_DEVICES"]="0"
    # 이미지 파일 받기
    image = request.files['image']
    # 선택된 가발 값 받기
    selected_wig = int(request.form['selectedWig'])

    source_path = './unprocessed/source.jpg'
    image.save(source_path)  # 이미지를 지정된 경로에 저장
        
    # 이곳에서 이미지 처리 및 가발 적용 로직을 구현
    script_path = "align_face.py"
        
    result1 = prepro_image(script_path)
    print(result1)
        
    # hair_synthesis.py 스크립트 실행
    hair_synthesis_script = "hair_synthesis.py"
    im_path1 = "source.png"
    im_path2 = "target" + str(selected_wig + 1) + ".png"
    print(im_path2)
    im_path3 = "source.png"
    sign = "realistic"
    smooth = "5"
        
    result2 = synthesis_image(hair_synthesis_script, im_path1, im_path2, im_path3, sign, smooth)
    print(result2)
        
    synthesis_output = "./output/source_target" + str(selected_wig + 1) + "_source_realistic.png"
    
    # 이미지 처리 완료 후, PNG 이미지를 열고 JPG로 변환
    with Image.open(synthesis_output) as img:
        img = img.convert('RGB')
        jpg_output = synthesis_output.replace(".png", ".jpg")
        img.save(jpg_output, "JPEG")

    jpg_path = "./output/source_target" + str(selected_wig + 1) + "_source_realistic.jpg"
    # JPG 파일을 클라이언트로 전송
    return send_file(jpg_path, mimetype='image/jpeg')

if __name__ == '__main__':
    app.run(host='192.168.35.53', port=5000, debug=False, use_reloader=False, threaded=True)

c:\jupyter\capston\bald
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.35.53:5000
Press CTRL+C to quit


True
전처리 완료
target3.png


192.168.35.53 - - [25/Oct/2023 23:15:54] "POST /wear_wig HTTP/1.1" 200 -


합성 완료


In [1]:
import os
import torch
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.models import load_model  # 변경된 부분
from flask import Flask, request, jsonify, send_file
import atexit
from database import save_data_to_database
from subprocess_code import prepro_image, synthesis_image
from inference import Predictor, CELEBRITY_LABELS, ANIMAL_LABELS

app = Flask(__name__)

# 모델폴더
model_folder = "./PredictModels"

# 이미지 전처리 함수
def preprocess_image(img_path):
    img = Image.open(img_path)
    img = img.resize((192, 192))
    img = img.convert("RGB")
    img = np.asarray(img)
    img = img.astype('float32') / 255
    return img

# 모델을 사용한 이미지 분류 함수
def predict_image_with_models(image_path, model_folder):
    categories = np.array(["0", "1", "2", "3"])
    scalp_category = ["미세각질", "피지과다", "모낭사이홍반", "모낭홍반/농포", "비듬", "탈모"]

    model_files = [f for f in os.listdir(model_folder) if f.endswith(".h5")]
    print(model_files)
    alter_predict = []

    predict1 = []
    predict2 = []
    predict3 = []
    predict4 = []
    predict5 = []
    predict6 = []
    image = Image.open(image_path)

    with tf.device("/device:CPU:0"):
        for i, model_file in enumerate(model_files):
            print(os.getcwd())
            print(model_file)
            model_path = model_folder + '/' + model_file
            print(model_path)
            model = load_model(model_path)
            test = []
            test.append(preprocess_image(image_path))
            test = np.array(test)

            
            predict = model.predict(test)
            predicted_class_index = predict[0]
            
            # Print class probabilities
            print(f"Probabilities for model {model_file} ({scalp_category[i]}):")
            for class_index, probability in enumerate(predicted_class_index):
                print(f'Class {class_index}: {probability * 100}%')
                [predict1, predict2, predict3, predict4, predict5, predict6][i].append(round(probability * 100, 2))
                
            print([predict1, predict2, predict3, predict4, predict5, predict6][i])
            predicted_class_index = np.argmax(predicted_class_index)
            predicted_category = categories[predicted_class_index]

            print(f"모델 파일: {model_file}")
            print("예측된 카테고리(" +  scalp_category[i] +  "): " +  predicted_category + "\n")

            alter_predict.append(predicted_category)

    print("alter_predict: ", alter_predict, "\n")
    
    # 피부진단 결과 계산
    skin_diagnosis = calculate_skin_diagnosis(alter_predict)

    return skin_diagnosis, alter_predict, predict1, predict2, predict3, predict4, predict5, predict6

def calculate_skin_diagnosis(alter_predict):
    # 여기서부터 피부진단 시작
    # 양호: 배열 안에 값이 전부 0인 경우
    if all(category == '0' for category in alter_predict):
        return "양호"

    # 건성: 배열 첫 번째 값은 3이고 다섯 번째 값이 1이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        alter_predict[4] == '1' and
        all(category != '3' for category in alter_predict[1:4] + alter_predict[5:])
    ):
        return "건성"

    # 지성: 배열 두 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[1] == '3' and
        all(category != '3' for category in alter_predict[2:])
    ):
        return "지성"

    # 민감성: 배열 세 번째 값이 3이고 나머지 값이 3이 아닌 경우
    if (
        alter_predict[2] == '3' and
        all(category != '3' for category in alter_predict[3:])
    ):
        return "민감성"

    # 지루성: 배열 첫 번째 값이 3이고 다섯 번째, 여섯 번째 값이 2~3이고 나머지가 3이 아닌 경우
    if (
        alter_predict[0] == '3' and
        (2 <= int(alter_predict[4]) <= 3) and
        (2 <= int(alter_predict[5]) <= 3) and
        all(category != '3' for category in alter_predict[1:4])
    ):
        return "지루성"

    # 염증성: 배열 네 번째 값이 3이고 세 번째 값이 2~3이면서 나머지가 3이 아닌 경우
    if (
        alter_predict[3] == '3' and
        (2 <= int(alter_predict[2]) <= 3)
    ):
        return "염증성"

    # 비듬성: 배열 2, 5번째 값이 2~3이고 나머지 배열 값이 3이 아닌 경우
    if (
        (2 <= int(alter_predict[1]) <= 3) and
        (2 <= int(alter_predict[4]) <= 3) and
        all(category != '3' for category in alter_predict[0:1] + alter_predict[2:4] + alter_predict[5:])
    ):
        return "비듬성"

    # 탈모: 배열 6번째 값이 3이고 나머지 배열 값이 3이 아닌 경우
    if (
        alter_predict[5] == '3' and
        all(category != '3' for category in alter_predict[:5])
    ):
        return "탈모"

    # 기타: 위 조건에 해당하지 않는 경우
    return "기타"

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'image' not in request.files:
        return jsonify({'error': 'No image part'})

    file = request.files['image']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        # 이미지를 저장할 경로를 지정합니다.
        upload_dir = './skin_upload'
        if not os.path.exists(upload_dir):
            os.makedirs(upload_dir)
        file_path = os.path.join(upload_dir, file.filename)
        file.save(file_path)

        # 이미지 분석 및 진단 결과 얻기
        diagnosis_result, alter_predict, predict1, predict2, predict3, predict4, predict5, predict6 = predict_image_with_models(file_path, model_folder)
        print(diagnosis_result)
        print(*alter_predict)
        predict1_str = ','.join(map(str, [int(round(x)) for x in predict1]))
        predict2_str = ','.join(map(str, [int(round(x)) for x in predict2]))
        predict3_str = ','.join(map(str, [int(round(x)) for x in predict3]))
        predict4_str = ','.join(map(str, [int(round(x)) for x in predict4]))
        predict5_str = ','.join(map(str, [int(round(x)) for x in predict5]))
        predict6_str = ','.join(map(str, [int(round(x)) for x in predict6]))

        print(predict1_str)
        print(predict2_str)
        print(predict3_str)
        print(predict4_str)
        print(predict5_str)
        print(predict6_str)
        # 진단 결과와 alter_predict를 함께 JSON 응답으로 반환
        response_data = {'message': diagnosis_result,'predict1': predict1_str, 
                        'predict2': predict2_str, 'predict3': predict3_str,
                        'predict4': predict4_str, 'predict5': predict5_str,
                        'predict6': predict6_str }
        
        # 데이터베이스에 데이터 저장
        save_data_to_database(file_path, diagnosis_result, alter_predict)

        print("Mysql 전송 완료")
        
        return jsonify(response_data)
    
selected_wig = 0 # default

@app.route('/wear_wig', methods=['POST'])
def wear_wig():
    print(torch.cuda.is_available())
    os.environ["CUDA_VISIBLE_DEVICES"]="0"
    # 이미지 파일 받기
    image = request.files['image']
    # 선택된 가발 값 받기
    selected_wig = int(request.form['selectedWig'])

    source_path = './unprocessed/source.jpg'
    image.save(source_path)  # 이미지를 지정된 경로에 저장
        
    # 이곳에서 이미지 처리 및 가발 적용 로직을 구현
    script_path = "align_face.py"
        
    result1 = prepro_image(script_path)
    print(result1)
        
    label = 'male'

    ref_path = './images/ref/target' + str(selected_wig+1) + '.png'
    src = Image.open('./images/src/source.png').convert("RGB")
    ref = Image.open(ref_path).convert("RGB")

    entity = "celebrity"

    predictor = Predictor(entity=entity)
    predictor.create_interpolation(label, src_image=src, ref_image=ref)
        
    synthesis_output = "./images/res.jpg"
    
    # 이미지 처리 완료 후, PNG 이미지를 열고 JPG로 변환
    with Image.open(synthesis_output) as img:
        img = img.convert('RGB')
        jpg_output = synthesis_output.replace(".png", ".jpg")
        img.save(jpg_output, "JPEG")

    jpg_path = "./images/res.jpg"
    # JPG 파일을 클라이언트로 전송
    return send_file(jpg_path, mimetype='image/jpeg')

@app.route('/progress_status', methods=['POST'])

    
if __name__ == '__main__':
    app.run(host='192.168.35.119', port=5000, debug=False, use_reloader=False, threaded=True)

c:\jupyter\capston\bald\hairskinWig
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.35.119:5000
Press CTRL+C to quit


['1_Corneous_Xception_1.h5', '2_Sebum_Xception_1.h5', '3_ErFo_Xception_1.h5', '4_Fep_Xception_1.h5', '5_Dandruff_Xception_1.h5', '6_Hairloss_Xception_1.h5']
c:\jupyter\capston\bald\hairskinWig
1_Corneous_Xception_1.h5
./PredictModels/1_Corneous_Xception_1.h5
Probabilities for model 1_Corneous_Xception_1.h5 (미세각질):
Class 0: 1.88522921718004e-06%
Class 1: 9.82828289270401%
Class 2: 1.5626650303602219%
Class 3: 88.60905766487122%
[0.0, 9.83, 1.56, 88.61]
모델 파일: 1_Corneous_Xception_1.h5
예측된 카테고리(미세각질): 3

c:\jupyter\capston\bald\hairskinWig
2_Sebum_Xception_1.h5
./PredictModels/2_Sebum_Xception_1.h5
Probabilities for model 2_Sebum_Xception_1.h5 (피지과다):
Class 0: 3.5286276938578665e-10%
Class 1: 99.99974966049194%
Class 2: 6.450978276006936e-05%
Class 3: 0.00019430765405559214%
[0.0, 100.0, 0.0, 0.0]
모델 파일: 2_Sebum_Xception_1.h5
예측된 카테고리(피지과다): 1

c:\jupyter\capston\bald\hairskinWig
3_ErFo_Xception_1.h5
./PredictModels/3_ErFo_Xception_1.h5
Probabilities for model 3_ErFo_Xception_1.h5 (모낭사이홍반

192.168.35.119 - - [02/Nov/2023 19:49:45] "POST /upload HTTP/1.1" 200 -


['1_Corneous_Xception_1.h5', '2_Sebum_Xception_1.h5', '3_ErFo_Xception_1.h5', '4_Fep_Xception_1.h5', '5_Dandruff_Xception_1.h5', '6_Hairloss_Xception_1.h5']
c:\jupyter\capston\bald\hairskinWig
1_Corneous_Xception_1.h5
./PredictModels/1_Corneous_Xception_1.h5
Probabilities for model 1_Corneous_Xception_1.h5 (미세각질):
Class 0: 3.665780786560324e-10%
Class 1: 94.08925175666809%
Class 2: 5.8773767203092575%
Class 3: 0.03336445370223373%
[0.0, 94.09, 5.88, 0.03]
모델 파일: 1_Corneous_Xception_1.h5
예측된 카테고리(미세각질): 1

c:\jupyter\capston\bald\hairskinWig
2_Sebum_Xception_1.h5
./PredictModels/2_Sebum_Xception_1.h5
Probabilities for model 2_Sebum_Xception_1.h5 (피지과다):
Class 0: 1.5578555334627708e-06%
Class 1: 98.13100099563599%
Class 2: 1.8673386424779892%
Class 3: 0.001660101770539768%
[0.0, 98.13, 1.87, 0.0]
모델 파일: 2_Sebum_Xception_1.h5
예측된 카테고리(피지과다): 1

c:\jupyter\capston\bald\hairskinWig
3_ErFo_Xception_1.h5
./PredictModels/3_ErFo_Xception_1.h5
Probabilities for model 3_ErFo_Xception_1.h5 (모낭사이홍반

192.168.35.119 - - [02/Nov/2023 19:51:59] "POST /upload HTTP/1.1" 200 -


['1_Corneous_Xception_1.h5', '2_Sebum_Xception_1.h5', '3_ErFo_Xception_1.h5', '4_Fep_Xception_1.h5', '5_Dandruff_Xception_1.h5', '6_Hairloss_Xception_1.h5']
c:\jupyter\capston\bald\hairskinWig
1_Corneous_Xception_1.h5
./PredictModels/1_Corneous_Xception_1.h5
Probabilities for model 1_Corneous_Xception_1.h5 (미세각질):
Class 0: 99.99994039535522%
Class 1: 2.151242739500958e-06%
Class 2: 5.678983256984793e-05%
Class 3: 5.592239171736679e-09%
[100.0, 0.0, 0.0, 0.0]
모델 파일: 1_Corneous_Xception_1.h5
예측된 카테고리(미세각질): 0

c:\jupyter\capston\bald\hairskinWig
2_Sebum_Xception_1.h5
./PredictModels/2_Sebum_Xception_1.h5
Probabilities for model 2_Sebum_Xception_1.h5 (피지과다):
Class 0: 99.9908447265625%
Class 1: 0.00217834585782839%
Class 2: 0.0069435162004083395%
Class 3: 2.3826287076644803e-05%
[99.99, 0.0, 0.01, 0.0]
모델 파일: 2_Sebum_Xception_1.h5
예측된 카테고리(피지과다): 0

c:\jupyter\capston\bald\hairskinWig
3_ErFo_Xception_1.h5
./PredictModels/3_ErFo_Xception_1.h5
Probabilities for model 3_ErFo_Xception_1.h5 (모낭

192.168.35.119 - - [02/Nov/2023 19:53:00] "POST /upload HTTP/1.1" 200 -
